In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#initiating training data and seperating them into training set and test sets 

reddit_data = pd.read_csv("reddit_train.csv") #train data
df = pd.DataFrame(reddit_data) #train data
reddit_data_copy = reddit_data.copy() #train data
reddit_data_copy = reddit_data_copy.drop(columns=["subreddits"]) #train data
X = reddit_data_copy[reddit_data_copy.columns[1]]  #raw comments

train_comments_list = []
for i in range(len(X.index)):
    train_comments_list.append(X.iloc[i])
train_comments_list = list(train_comments_list)
train_comments_list = np.array(train_comments_list)


Y = reddit_data[reddit_data.columns[-1]]
y_modified = []         #raw classes
train_subreddits_class = []    #important, class in integer
for i in range(len(Y.index)):
    y_modified.append(Y.loc[i])
for i in range(len(y_modified)):
    if y_modified[i] == 'hockey':
        train_subreddits_class.append(1)
    elif y_modified[i] == 'nba':
        train_subreddits_class.append(2)
    elif y_modified[i] == 'leagueoflegends':
        train_subreddits_class.append(3)
    elif y_modified[i] == 'soccer':
        train_subreddits_class.append(4)
    elif y_modified[i] == 'funny':
        train_subreddits_class.append(5)
    elif y_modified[i] == 'movies':
        train_subreddits_class.append(6)
    elif y_modified[i] == 'Overwatch':
        train_subreddits_class.append(7)
    elif y_modified[i] == 'anime':
        train_subreddits_class.append(8)
    elif y_modified[i] == 'trees':
        train_subreddits_class.append(9)
    elif y_modified[i] == 'GlobalOffensive':
        train_subreddits_class.append(10)
    elif y_modified[i] == 'nfl':
        train_subreddits_class.append(11)
    elif y_modified[i] == 'AskReddit':
        train_subreddits_class.append(12)
    elif y_modified[i] == 'gameofthrones':
        train_subreddits_class.append(13)
    elif y_modified[i] == 'conspiracy':
        train_subreddits_class.append(14)
    elif y_modified[i] == 'worldnews':
        train_subreddits_class.append(15)
    elif y_modified[i] == 'wow':
        train_subreddits_class.append(16)
    elif y_modified[i] == 'europe':
        train_subreddits_class.append(17)
    elif y_modified[i] == 'canada':
        train_subreddits_class.append(18)
    elif y_modified[i] == 'Music':
        train_subreddits_class.append(19)
    elif y_modified[i] == 'baseball':
        train_subreddits_class.append(20)
train_subreddits_class = np.array(train_subreddits_class)
X_train = train_comments_list
y_train = train_subreddits_class

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.preprocessing import normalize
import re

#preprocess and normalize data by lammatizing, removing punctuation and stems etc. 

lem = WordNetLemmatizer()
tknzr = WordPunctTokenizer()
stem = PorterStemmer()
replace_by_space = re.compile('[/(){}\[\]\|@,;!.#><?""''%$^*`:]')
stop_words = set(stopwords.words('english'))
unused_char = re.compile('[^0-9a-zA-Z]')

for i in range(len(X_train)):
    X_train[i] = X_train[i].lower()
    X_train[i] = replace_by_space.sub(' ', X_train[i])
    X_train[i] = unused_char.sub(' ', X_train[i])
    X_train[i] = ' '.join(lem.lemmatize(word) for word in X_train[i].split() if word not in stop_words)

tf_idf_vectorizer = TfidfVectorizer(stop_words = set(stopwords.words('english')), max_df = 1.0, ngram_range=(1, 3), min_df = 0.000005, analyzer='word',binary=True, tokenizer = WordPunctTokenizer().tokenize)
vectors_train_idf = tf_idf_vectorizer.fit_transform(X_train)
terms = tf_idf_vectorizer.get_feature_names()
vectors_train_nomalized = normalize(vectors_train_idf)

/Users/theone/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'"] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [15]:
#feature selection
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression, mutual_info_classif
skb = SelectKBest(chi2, k=70000)
vectors_train_idf_normalized_selected = skb.fit_transform(vectors_train_nomalized, y_train)

In [16]:
# splitting training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors_train_idf_normalized_selected, y_train, train_size=0.9, test_size=0.1)

In [17]:
#Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score

clf_nb = MultinomialNB(alpha=0.001)
clf_nb.fit(X_train, y_train)
y_pred = clf_nb.predict(X_test)

#printing the predictions
#print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.70      0.67      0.69       348
           2       0.72      0.70      0.71       351
           3       0.75      0.75      0.75       336
           4       0.72      0.65      0.68       327
           5       0.33      0.36      0.34       346
           6       0.58      0.67      0.62       357
           7       0.80      0.73      0.77       337
           8       0.73      0.73      0.73       367
           9       0.56      0.59      0.58       367
          10       0.86      0.67      0.75       374
          11       0.70      0.69      0.69       360
          12       0.30      0.38      0.34       336
          13       0.93      0.68      0.78       354
          14       0.40      0.50      0.45       334
          15       0.46      0.37      0.41       366
          16       0.73      0.77      0.75       366
          17       0.55      0.62      0.58       331
          18       0.46    

In [37]:
#linear model
from sklearn import linear_model
class_weights = {1: 0.9, 2:0.7, 3:0.7, 4:0.9, 5:1.5, 6:0.9, 7:0.5, 8:0.9, 9:1, 10:0.8, 11:0.9, 12:1.5, 13:0.3, 14:1.2, 15:1.2, 16:0.5, 17:1.2, 18:1.2, 19:0.5, 20:0.9}

clf_SDG = linear_model.SGDClassifier(max_iter=1000, tol=1e-3, class_weight=class_weights, penalty='l2', alpha=0.0001, random_state=0)
clf_SDG.fit(X_train, y_train)
y_pred = clf_SDG.predict(X_test)

#printing the predictions
#print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.70      0.62      0.66       326
           2       0.81      0.68      0.74       370
           3       0.72      0.74      0.73       333
           4       0.61      0.66      0.63       340
           5       0.28      0.28      0.28       353
           6       0.59      0.61      0.60       352
           7       0.84      0.72      0.77       331
           8       0.66      0.70      0.68       374
           9       0.58      0.63      0.60       361
          10       0.69      0.60      0.64       345
          11       0.58      0.65      0.61       335
          12       0.28      0.30      0.29       343
          13       0.88      0.69      0.77       342
          14       0.51      0.54      0.53       366
          15       0.46      0.26      0.34       356
          16       0.77      0.73      0.75       362
          17       0.40      0.67      0.50       353
          18       0.46    

In [29]:
#Linear SVC
from sklearn.svm import LinearSVC
class_weights = {1: 0.9, 2:0.7, 3:0.7, 4:0.9, 5:1.5, 6:0.9, 7:0.5, 8:0.9, 9:1, 10:0.8, 11:0.9, 12:1.5, 13:0.1, 14:1.2, 15:1.2, 16:0.1, 17:1.2, 18:1.2, 19:0.5, 20:0.9}
clf_svc = LinearSVC(random_state=0, tol=1e-5, class_weight=class_weights)
clf_svc.fit(X_train, y_train)
y_pred = clf_svc.predict(X_test)

#printing the predictions
#print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.68      0.62      0.65       326
           2       0.84      0.65      0.73       370
           3       0.73      0.72      0.73       333
           4       0.70      0.64      0.67       340
           5       0.22      0.36      0.27       353
           6       0.63      0.57      0.60       352
           7       0.84      0.70      0.77       331
           8       0.65      0.66      0.66       374
           9       0.61      0.62      0.61       361
          10       0.69      0.65      0.67       345
          11       0.68      0.62      0.65       335
          12       0.22      0.43      0.29       343
          13       0.98      0.53      0.69       342
          14       0.46      0.51      0.48       366
          15       0.36      0.39      0.37       356
          16       0.96      0.44      0.61       362
          17       0.53      0.60      0.56       353
          18       0.46    

In [30]:
#MLP
#need to manually terminate this cell after 3/4 minutes
#if an error appears, wait longer 
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
clf_mlp = MLPClassifier(alpha = 0.00001, learning_rate = 'adaptive', learning_rate_init = 0.005, early_stopping = True, max_iter = 100, activation = 'relu', random_state=0)
clf_mlp.fit(X_train, y_train)
y_pred = clf_mlp.predict(X_test)

#printing the predictions
#print(metrics.classification_report(y_test, y_pred))

/Users/yoooooooh/Desktop/Anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


              precision    recall  f1-score   support

           1       0.75      0.65      0.70       326
           2       0.77      0.72      0.75       370
           3       0.74      0.74      0.74       333
           4       0.77      0.65      0.70       340
           5       0.24      0.30      0.27       353
           6       0.62      0.62      0.62       352
           7       0.76      0.76      0.76       331
           8       0.70      0.69      0.70       374
           9       0.64      0.62      0.63       361
          10       0.77      0.62      0.69       345
          11       0.63      0.67      0.65       335
          12       0.29      0.40      0.34       343
          13       0.85      0.75      0.80       342
          14       0.50      0.50      0.50       366
          15       0.37      0.49      0.42       356
          16       0.75      0.79      0.77       362
          17       0.60      0.57      0.59       353
          18       0.61    

In [17]:
#initializing test data set
reddit_data_test = pd.read_csv["reddit_test.csv"]
df = pd.DataFrame(reddit_data_test)
X = reddit_data_test[reddit_data_test.columns[1]]
train_comments_list = []
for i in range(len(X.index)):
    train_comments_list.append(X.iloc[i])
train_comments_list = np.array(train_comments_list)

for i in range(len(train_comments_list)):
    train_comments_list[i] = train_comments_list[i].lower()
    train_comments_list[i] = replace_by_space.sub(' ', train_comments_list[i])
    train_comments_list[i] = unused_char.sub(' ', train_comments_list[i])
    train_comments_list[i] = ' '.join(lem.lemmatize(word) for word in train_comments_list[i].split() if word not in stop_words)

vectors_real_test_idf = tf_idf_vectorizer.transform(train_comments_list)
vectors_real_test_idf_selected_nomalized = normalize(vectors_real_test_idf)
vectors_real_test_idf_selected = skb.transform(vectors_real_test_idf_selected_nomalized)

y_pred1 = clf_nb.predict(vectors_real_test_idf_selected)
y_pred2 = clf_svc.predict(vectors_real_test_idf_selected)
y_pred3 = clf_mlp.predict(vectors_real_test_idf_selected)
y_pred4 = clf_SDG.predict(vectors_real_test_idf_selected)

#outputs the final prediction using hard voting classifier: seeing which prediction of the class is most agreed among the different models 
y_pred = []
for i in range(len(y_pred1)):
    if y_pred1[i] == y_pred2[i] == y_pred3[i] == y_pred4[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] == y_pred2[i] == y_pred3[i] != y_pred4[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] == y_pred3[i] == y_pred4[i] != y_pred2[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] == y_pred2[i] == y_pred4[i] != y_pred3[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] != y_pred2[i] == y_pred3[i] == y_pred4[i]:
        y_pred.append(y_pred4[i])
        
    elif y_pred1[i] == y_pred2[i] != y_pred3[i] == y_pred4[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] == y_pred3[i] != y_pred2[i] == y_pred4[i]:
        y_pred.append(y_pred1[i])   
        
    elif y_pred1[i] == y_pred4[i] != y_pred2[i] == y_pred3[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] == y_pred2[i] != y_pred3[i] != y_pred4[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] == y_pred3[i] != y_pred4[i] != y_pred2[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred1[i] == y_pred4[i] != y_pred2[i] != y_pred3[i]:
        y_pred.append(y_pred1[i])
        
    elif y_pred2[i] == y_pred3[i] != y_pred1[i] != y_pred4[i]:
        y_pred.append(y_pred2[i])    
        
    elif y_pred2[i] == y_pred4[i] != y_pred1[i] != y_pred3[i]:
        y_pred.append(y_pred2[i]) 
        
    elif y_pred3[i] == y_pred4[i] != y_pred1[i] != y_pred2[i]:
        y_pred.append(y_pred3[i])
    
    elif y_pred3[i] != y_pred4[i] != y_pred1[i] != y_pred2[i]:
        y_pred.append(y_pred1[i])
        
        
        
#change classes back to string format
y_pred_modified = []
for i in range(len(y_pred)):        #raw classes
    if y_pred[i] == 1:
        y_pred_modified.append('hockey')
    elif y_pred[i] == 2:
        y_pred_modified.append('nba')
    elif y_pred[i] == 3:
        y_pred_modified.append('leagueoflegends')
    elif y_pred[i] == 4:
        y_pred_modified.append('soccer')
    elif y_pred[i] == 5:
        y_pred_modified.append('funny')
    elif y_pred[i] == 6:
        y_pred_modified.append('movies')
    elif y_pred[i] == 7:
        y_pred_modified.append('Overwatch')
    elif y_pred[i] == 8:
        y_pred_modified.append('anime')
    elif y_pred[i] == 9:
        y_pred_modified.append('trees')
    elif y_pred[i] == 10:
        y_pred_modified.append('GlobalOffensive')
    elif y_pred[i] == 11:
        y_pred_modified.append('nfl')
    elif y_pred[i] == 12:
        y_pred_modified.append('AskReddit')
    elif y_pred[i] == 13:
        y_pred_modified.append('gameofthrones')
    elif y_pred[i] == 14:
        y_pred_modified.append('conspiracy')
    elif y_pred[i] == 15:
        y_pred_modified.append('worldnews')
    elif y_pred[i] == 16:
        y_pred_modified.append('wow')
    elif y_pred[i] == 17:
        y_pred_modified.append('europe')
    elif y_pred[i] == 18:
        y_pred_modified.append('canada')
    elif y_pred[i] == 19:
        y_pred_modified.append('Music')
    elif y_pred[i] == 20:
        y_pred_modified.append('baseball')
y_pred_modified = np.array(y_pred_modified)

['nfl' 'europe' 'anime' ... 'GlobalOffensive' 'gameofthrones' 'wow']
30000


In [18]:
#output the prediction results as a csv file. Please rename the file manually if needed
np.savetxt("prediction_voted4.csv", np.dstack((np.arange(0, y_pred_modified.size),y_pred_modified))[0],"%s,%s",header="Id,Category")

NameError: name 'y_pred_modified' is not defined